In [364]:
from elasticsearch import Elasticsearch
from datetime import datetime, timedelta

import json
cluster = ['ip-172-31-0-107', 'ip-172-31-0-100', 'ip-172-31-0-105', 'ip-172-31-0-106']
es = Elasticsearch(cluster, port=9200)

In [365]:
def dispatch(ctime, location, driver, name, p1=None, p2=None):
        #d = retrieveDriver(driver, es)
        
        p = scanPassenger(location, es)
        
        dDoc = {"ctime": ctime, "location": location, "status": "idle"}
        if p:
            doc = {"status": "pickup", "driver": driver, "ctime": ctime}
            updatePassenger(p['id'], doc, es)

            if p1: 
                dDoc['p2'] = p['id']
            else:
                dDoc['p1'] = p['id']
                
            dDoc['status'] = "pickup"
            dDoc['destination'] = p['location']
            dDoc['altdest1'] = p['id']
            

        updateDriver(driver, dDoc, es)
        
        bulk = (1, '{{doc: {}}}'.format(json.dumps(dDoc)))
        return (bulk)

In [ ]:
      'driver': {'type': 'string', "include_in_all": "false"},
        'destination': {'type': 'geo_point', 'lat_lon': 'true', "include_in_all": "false"},
        'destinationid':{'type': 'string', "include_in_all": "false"},                
        'altdest1': {'type': 'geo_point', 'lat_lon': 'true', "include_in_all": "false"},
        'altdestid1':{'type': 'string', "include_in_all": "false"},                    
        'altdest2': {'type': 'geo_point', 'lat_lon': 'true', "include_in_all": "false"},
        'altdestid2':{'type': 'string', "include_in_all": "false"}, 
        'origin': {'type': 'geo_point', 'lat_lon': 'true', "include_in_all": "false"},
        'path': {'type': 'string', "include_in_all": "false"}


In [472]:
window = 'now-2h'
location =[40.7488,-73.9854]
q = {"query" : {
    "bool" : {
        "must" : {
            "term" : { "status" : "arrived" }
        },
        "filter": {
            "geo_distance": {
                "distance": '20km',
                "distance_type": "plane", 
                "location": location }},
    
        "should" : [
            {
                "terms" : { "destinationid" : ["Empire Building", "Times Square","Intrepid Sea, Air & Space Museum", "Metropolitan Opera House", "American Museum of Natural History"] }
            },
            {
                "terms" : { "altdest1id" : ["Times Square","Intrepid Sea, Air & Space Museum", "Empire Building", "Metropolitan Opera House"] }
            },
            {
                "terms" : { "altdest2id" : ["9/11 Memorial & Museum", "Intrepid Sea, Air & Space Museum", "Times Square", "Empire Building", "Metropolitan Opera House"] }
            }
        ],
        "minimum_should_match" : 0,
        "boost" : 1.0
    }
}}

res = es.search(index='passenger', doc_type='rolling', body=q, ignore=[404, 400])



In [479]:
def retrieveDriver(driver, es):
    _ = es.get(index='driver', doc_type='rolling', id=driver, ignore=[404, 400])
    return _
  

In [480]:
res = retrieveDriver(690990909, es)

In [481]:
res

{u'_id': u'690990909',
 u'_index': u'driver',
 u'_type': u'rolling',
 u'found': False}